In [ ]:
import pandas as pd
import json
import re
import numpy as np
import torch
from torch_geometric.data import InMemoryDataset, download_url
import networkx as nx
from networkx.algorithms import community
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from pyvis.network import Network
import string
from colorama import Fore
import termtables as tt
import seaborn as sns

In [ ]:
with open("./Data/Net-content.json","r", encoding='utf-8') as f:
    re_num = json.load(f)

In [ ]:
class Demo_Data(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
    #file
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
   
    @property
    def processed_file_names(self):
        return ['data.pt']
    # download from website
    # def download(self):
    #     # Download to `self.raw_dir`.
    #     download_url(url, self.raw_dir)
        ...
    #data generate
    def process(self):
        # Read data into huge `Data` list.
        # Read data into huge `Data` list.
        # building
        
        start=0
        end=len(list_source)
        
        Edge_index = torch.tensor([list_source,
                                   list_target], dtype=torch.long)
        
        # s=list_source[start:end].copy()
        # s.extend(list_target[start:end])
        print(len(re_num))
        
        # node_num=len(set(s))

        # node feature
        sample_feature=[0]*len(re_num)
        X = torch.tensor(np.array(sample_feature).reshape(-1, 1).tolist(), dtype=torch.float)
        # node label
        # Y = torch.tensor([0,1,0],dtype=torch.float)
        
        #building
        # data = Data(x=X, edge_index=Edge_index, y=Y)
        data = Data(x=X,edge_index=Edge_index)
        
        # put in datalist
        data_list = [data]

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        data.num_nodes = len(re_num)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
dataset = Demo_Data("Whole_Network")
data=dataset[0]
dataset.get_summary()

In [ ]:
# Display 
G = to_networkx(data, to_undirected=False)# Directed
degrees = [val for (node, val) in G.degree()]
display(pd.DataFrame(pd.Series(degrees).describe()).transpose().round(2))
print(len(degrees))
print(sum(degrees))
plt.figure(figsize=(10, 6))
# plt.hist(degrees, bins=100)
# plt.hist(degrees, range(0,10))
plt.hist(degrees, range(0,50),rwidth=0.5,align="left")

plt.xlabel("node degree")
plt.show()

# Similarity

In [ ]:
#nltk_word
def word_div(strings):
    strings=strings.lower()
    strings=strings.translate(str.maketrans('', '', string.punctuation))
    token=nltk.word_tokenize(strings)
    stop_words = stopwords.words('english')
    tokens=[]
    for i_2 in range(len(token)):
        if token[i_2] not in stop_words:
            tokens.append(token[i_2])   
    filter_word=['from', 'subject', 're', 'edu', 'use','introduction','conclusion','evaluation','figure','table','survey','et','al','research','we','use','work','issue','also','future','however','finally','task','tasks','paper','nlp','recent','natural','language','processing','nevertheless']
    clean_token=[]
    for item in tokens:
        item=lemmatizer.lemmatize(item, pos="n")
        if item not in filter_word and re.search(r'^\d+$',item) is None:
            # clean_token.append(st.stem(item))
            clean_token.append(item)
    return clean_token

In [ ]:
#Similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from sentence_transformers import SentenceTransformer, util
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
lemmatizer = WordNetLemmatizer()
st = LancasterStemmer()
# model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('scibert_scivocab_uncased')
# model = SentenceTransformer('paraphrase-MiniLM-L3-v2')
sentences=[]
strings_total=''
branch=2
pair_list=[]
pair_list_total=[]
for item in re_num:
    if 'conclusion' not in re_num[item].keys():
        # print(re_num[item]['num'])
        continue
    
    sentences.append(re_num[item]['conclusion'])
    strings_total=strings_total+re_num[item]['conclusion']
    


#Compute embeddings
embeddings = model.encode(sentences, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i_1 in range(len(cosine_scores)-1):
    for j_1 in range(i_1+1, len(cosine_scores)):
        pairs.append({'index': [i_1, j_1], 'score': cosine_scores[i_1][j_1]})

#Sort scores in decreasing order
pairs_dec = sorted(pairs, key=lambda x: x['score'], reverse=True)

#Simliarity of each article (n-branch)
start=0
for i in range(1,len(pairs)):
    strings_pair=''
    flag=pairs[i-1]['index'][0]
    i_1, j_1 = pairs[i]['index']
    if i_1!=flag:
        pair=pairs[start:i]
        pair = sorted(pair, key=lambda x: x['score'], reverse=True)
        # for building tree
        for j in range(branch):
            pair[j]['s1']=sentences[pair[j]['index'][0]]
            pair[j]['s2']=sentences[pair[j]['index'][1]]
            strings_pair=sentences[pair[j]['index'][0]]+sentences[pair[j]['index'][1]]
            # print(Fore.BLACK + "{} \t\t {} \t\t Score: {:.4f}".format(sentences[pair[j]['index'][0]], sentences[pair[j]['index'][1]], pair[j]['score']))

            #nltk->top:10
            clean_token=word_div(strings_pair)
            freq_dist = nltk.FreqDist(clean_token)
            # print(Fore.BLUE + 'hypernym:',freq_dist.most_common(10))
            pair[j]['hypernym']=freq_dist.most_common(10)
            
            #nltk ->common_word
            S_0=word_div(sentences[pair[j]['index'][0]])
            S_1=word_div(sentences[pair[j]['index'][1]])
            and_list=set(S_0)&set(S_1)
            # print(Fore.MAGENTA + 'common_word:',and_list)
            pair[j]['common_word']=and_list

            #add data to list
            pair_list.append(pair[j])

        start=i


In [ ]:
for j in range(len(pairs)):
    pairs[j]['s1']=sentences[pairs[j]['index'][0]]
    pairs[j]['s2']=sentences[pairs[j]['index'][1]]
    strings_pair=sentences[pairs[j]['index'][0]]+sentences[pairs[j]['index'][1]]
    # print(Fore.BLACK + "{} \t\t {} \t\t Score: {:.4f}".format(sentences[pair[j]['index'][0]], sentences[pair[j]['index'][1]], pair[j]['score']))

    #nltk->top:10
    clean_token=word_div(strings_pair)
    freq_dist = nltk.FreqDist(clean_token)
    # print(Fore.BLUE + 'hypernym:',freq_dist.most_common(10))
    pairs[j]['hypernym']=freq_dist.most_common(10)

    #nltk -> common word 
    S_0=word_div(sentences[pairs[j]['index'][0]])
    S_1=word_div(sentences[pairs[j]['index'][1]])
    and_list=set(S_0)&set(S_1)
    # print(Fore.MAGENTA + 'common_word:',and_list)
    pairs[j]['common_word']=and_list


In [ ]:
pairs

In [ ]:
pair_list